In this notebook we will make a summary of the results thus far. We start by replicating the results from in the paper for trine measurement where we make use equation A10, which is equivalent to integrating over the Haar measure.

In [1]:
import sympy as sp
import numpy as np
from sympy.matrices import Matrix
from sympy.physics.quantum import TensorProduct, Dagger
import matplotlib.pyplot as plt

from core import POVMProtocolCalculator
from utils import eigen_value_measure_mathematica, cyclic_symmetry, number_to_base, optimal_eig_new_measure, optimal_eig_old_measure, simplified_root_mean_square_measure_old_with_z, simplified_root_mean_square_measure_old_without_z, simplified_root_mean_square_measure_new 

In [2]:
phi0 = Matrix([1, 0])
phi1 = Matrix([sp.nsimplify(1/2), sp.nsimplify(sp.sqrt(3)/2)])
phi2 = Matrix([sp.nsimplify(1/2), sp.nsimplify(-sp.sqrt(3)/2)])

M0 = sp.nsimplify(2 / 3) * phi0 * Dagger(phi0)
M1 = sp.nsimplify(2 / 3) * phi1 * Dagger(phi1)
M2 = sp.nsimplify(2 / 3) * phi2 * Dagger(phi2)

povm_trine = POVMProtocolCalculator(2, M0, M1, M2)

In [3]:
povm_trine.calculate_measures_sympy(simplified_root_mean_square_measure_old_with_z)

Calculating 512 measures...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:01<00:00, 384.39it/s]

Done!


In [4]:
povm_trine.lowest_measures

[ProtocolResult(protocol=frozenset({(0, 1), (0, 2), (0, 0), (1, 0), (2, 0)}), measure_function='simplified_root_mean_square_measure_old_with_z', measure_value=0.0641500299099585)]

As expected there is one lowest measure which is the same as the result from the paper {(0, 1), (0, 2), (0, 0), (1, 0), (2, 0)} and the measure has a value of $ \epsilon = \frac{1}{9\sqrt{3}} \approx 0.064 $.

We can check that this is still the same protocol if we remove the $ |z|^2 $ from equation A10 to get equation A11  

In [5]:
povm_trine.calculate_measures_sympy(simplified_root_mean_square_measure_old_without_z)

Calculating 512 measures...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:00<00:00, 3814.85it/s]

Done!


In [6]:
povm_trine.lowest_measures

[ProtocolResult(protocol=frozenset({(0, 1), (0, 2), (0, 0), (1, 0), (2, 0)}), measure_function='simplified_root_mean_square_measure_old_without_z', measure_value=0.0641500299099585)]

Since this protocol doesn't have any complex components the measure is the same as before $ \epsilon = 0.064 $

We now use the optimisation procedure laid out in section B to see if we can improve on the above

In [ ]:
povm_trine.calculate_measures_mathematica()